In [75]:
# importar paquetes a ocupar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pycaret
import itertools

from pycaret.classification import *

In [76]:
model = load_model('models/lightgbm_less_columns')

Transformation Pipeline and Model Successfully Loaded


In [77]:
brawlers_stats = pd.read_csv('datasets/brawlers/brawlers_stats.csv', index_col=0)
players = pd.read_csv('datasets/players/players.csv', index_col=0)
battlelog = pd.read_csv('datasets/teams/battlelog_teams.csv', index_col=0)

In [78]:
mode_map = battlelog[['event.mode','event.map']].drop_duplicates().reset_index(drop=True)
mode_map

,event.mode,event.map
0,brawlBall,Center Stage
1,heist,Pit Stop
2,gemGrab,Double Swoosh
3,gemGrab,Minecart Madness
4,knockout,Belle's Rock
5,brawlBall,Backyard Bowl
6,knockout,Goldarm Gulch
7,bounty,Hideout
8,bounty,Shooting Star
9,brawlBall,Sneaky Fields


In [79]:
avg_rank = players[['trophies','highestTrophies','expLevel']].agg(np.mean)
avg_rank

trophies           32622.569317
highestTrophies    33130.258136
expLevel             211.054804
dtype: float64

In [80]:
avg_trophies = pd.concat([
	battlelog['battle.team1.player1.brawler.trophies'], 
	battlelog['battle.team1.player2.brawler.trophies'], 
	battlelog['battle.team1.player3.brawler.trophies'], 
	battlelog['battle.team2.player1.brawler.trophies'], 
	battlelog['battle.team2.player2.brawler.trophies'], 
	battlelog['battle.team2.player3.brawler.trophies']]).agg(np.mean)
avg_trophies

623.2829978068022

In [81]:
combinations = pd.DataFrame(itertools.combinations(brawlers_stats['name'], 3))
combinations.columns = ['battle.player1.brawler.name', 'battle.player2.brawler.name', 'battle.player3.brawler.name']
combinations

,battle.player1.brawler.name,battle.player2.brawler.name,battle.player3.brawler.name
0,SHELLY,COLT,BULL
1,SHELLY,COLT,BROCK
2,SHELLY,COLT,RICO
3,SHELLY,COLT,SPIKE
4,SHELLY,COLT,BARLEY
...,...,...,...
41659,GUS,GRAY,MANDY
41660,BUSTER,CHESTER,GRAY
41661,BUSTER,CHESTER,MANDY
41662,BUSTER,GRAY,MANDY


In [82]:
predictor = pd.DataFrame()

for i, item in enumerate(mode_map['event.mode']):
	combinations['event.mode'] = mode_map.loc[i, 'event.mode']
	combinations['event.map'] = mode_map.loc[i, 'event.map']
	predictor = pd.concat([predictor, combinations], ignore_index=True)

predictor.reset_index(drop=True)
predictor

,battle.player1.brawler.name,battle.player2.brawler.name,battle.player3.brawler.name,event.mode,event.map
0,SHELLY,COLT,BULL,brawlBall,Center Stage
1,SHELLY,COLT,BROCK,brawlBall,Center Stage
2,SHELLY,COLT,RICO,brawlBall,Center Stage
3,SHELLY,COLT,SPIKE,brawlBall,Center Stage
4,SHELLY,COLT,BARLEY,brawlBall,Center Stage
...,...,...,...,...,...
1791547,GUS,GRAY,MANDY,hotZone,Parallel Plays
1791548,BUSTER,CHESTER,GRAY,hotZone,Parallel Plays
1791549,BUSTER,CHESTER,MANDY,hotZone,Parallel Plays
1791550,BUSTER,GRAY,MANDY,hotZone,Parallel Plays


In [83]:
predictor['battle.mode'] = predictor['event.mode']
predictor['battle.type'] = 'ranked'
predictor['battle.player1.trophies'] = avg_rank.trophies
predictor['battle.player1.brawler.trophies'] = avg_trophies
predictor['battle.player1.brawler.power'] = 11
predictor['battle.player2.trophies'] = avg_rank.trophies
predictor['battle.player2.brawler.trophies'] = avg_trophies
predictor['battle.player2.brawler.power'] = 11
predictor['battle.player3.trophies'] = avg_rank.trophies
predictor['battle.player3.brawler.trophies'] = avg_trophies
predictor['battle.player3.brawler.power'] = 11

In [84]:
predictor = predictor.merge(brawlers_stats.add_prefix('battle.player1.brawler.'), on='battle.player1.brawler.name')
predictor = predictor.merge(brawlers_stats.add_prefix('battle.player2.brawler.'), on='battle.player2.brawler.name')
predictor = predictor.merge(brawlers_stats.add_prefix('battle.player3.brawler.'), on='battle.player3.brawler.name')

In [85]:
predictor

,battle.player1.brawler.name,battle.player2.brawler.name,battle.player3.brawler.name,event.mode,event.map,battle.mode,battle.type,battle.player1.trophies,battle.player1.brawler.trophies,battle.player1.brawler.power,battle.player2.trophies,battle.player2.brawler.trophies,battle.player2.brawler.power,battle.player3.trophies,battle.player3.brawler.trophies,battle.player3.brawler.power,battle.player1.brawler.id,battle.player1.brawler.role,battle.player1.brawler.range,battle.player1.brawler.speed,battle.player1.brawler.health,battle.player2.brawler.id,battle.player2.brawler.role,battle.player2.brawler.range,battle.player2.brawler.speed,battle.player2.brawler.health,battle.player3.brawler.id,battle.player3.brawler.role,battle.player3.brawler.range,battle.player3.brawler.speed,battle.player3.brawler.health
0,SHELLY,COLT,BULL,brawlBall,Center Stage,brawlBall,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000000,tank stopper,long,normal,5320,16000001,ranged dps,long,normal,3920,16000002,close quarters,short,fast,7000
1,SHELLY,COLT,BULL,heist,Pit Stop,heist,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000000,tank stopper,long,normal,5320,16000001,ranged dps,long,normal,3920,16000002,close quarters,short,fast,7000
2,SHELLY,COLT,BULL,gemGrab,Double Swoosh,gemGrab,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000000,tank stopper,long,normal,5320,16000001,ranged dps,long,normal,3920,16000002,close quarters,short,fast,7000
3,SHELLY,COLT,BULL,gemGrab,Minecart Madness,gemGrab,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000000,tank stopper,long,normal,5320,16000001,ranged dps,long,normal,3920,16000002,close quarters,short,fast,7000
4,SHELLY,COLT,BULL,knockout,Belle's Rock,knockout,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000000,tank stopper,long,normal,5320,16000001,ranged dps,long,normal,3920,16000002,close quarters,short,fast,7000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791547,CHESTER,GRAY,MANDY,knockout,X Marks the Spot,knockout,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000063,assassin,long,fast,4785,16000064,sniper,long,normal,4785,16000065,sniper,very long,normal,5600
1791548,CHESTER,GRAY,MANDY,knockout,New Perspective,knockout,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000063,assassin,long,fast,4785,16000064,sniper,long,normal,4785,16000065,sniper,very long,normal,5600
1791549,CHESTER,GRAY,MANDY,heist,Tornado Ring,heist,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000063,assassin,long,fast,4785,16000064,sniper,long,normal,4785,16000065,sniper,very long,normal,5600
1791550,CHESTER,GRAY,MANDY,hotZone,Split,hotZone,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000063,assassin,long,fast,4785,16000064,sniper,long,normal,4785,16000065,sniper,very long,normal,5600


In [86]:
predictions = predict_model(model, data = predictor, raw_score=True)

In [90]:
predictions['Label'].value_counts()

n    1791552
Name: Label, dtype: int64

In [89]:
predictions.sort_values(by='Score_y', ascending=False)

#SE DEBE ELIMINAR TROPHIES DEL MODELO

,battle.player1.brawler.name,battle.player2.brawler.name,battle.player3.brawler.name,event.mode,event.map,battle.mode,battle.type,battle.player1.trophies,battle.player1.brawler.trophies,battle.player1.brawler.power,battle.player2.trophies,battle.player2.brawler.trophies,battle.player2.brawler.power,battle.player3.trophies,battle.player3.brawler.trophies,battle.player3.brawler.power,battle.player1.brawler.id,battle.player1.brawler.role,battle.player1.brawler.range,battle.player1.brawler.speed,battle.player1.brawler.health,battle.player2.brawler.id,battle.player2.brawler.role,battle.player2.brawler.range,battle.player2.brawler.speed,battle.player2.brawler.health,battle.player3.brawler.id,battle.player3.brawler.role,battle.player3.brawler.range,battle.player3.brawler.speed,battle.player3.brawler.health,Label,Score_n,Score_y
1784754,ROSA,BUSTER,MANDY,knockout,New Perspective,knockout,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000024,tank,short,fast,7830,16000062,area control,normal,fast,6960,16000065,sniper,very long,normal,5600,n,0.5334,0.4666
1782217,ROSA,GUS,MANDY,knockout,New Perspective,knockout,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000024,tank,short,fast,7830,16000061,healer,long,normal,4480,16000065,sniper,very long,normal,5600,n,0.5380,0.4620
1784566,FRANK,BUSTER,MANDY,brawlBall,Beach Ball,brawlBall,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000020,tank,normal,fast,10150,16000062,area control,normal,fast,6960,16000065,sniper,very long,normal,5600,n,0.5451,0.4549
1785872,ASH,BUSTER,MANDY,knockout,New Perspective,knockout,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000051,tank,short,normal,7830,16000062,area control,normal,fast,6960,16000065,sniper,very long,normal,5600,n,0.5481,0.4519
1786216,SAM,BUSTER,MANDY,knockout,New Perspective,knockout,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000060,tank,short,fast,7560,16000062,area control,normal,fast,6960,16000065,sniper,very long,normal,5600,n,0.5481,0.4519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23973,MORTIS,BO,PIPER,hotZone,Open Zone,hotZone,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000011,assassin,short,very fast,5700,16000014,area control,long,normal,5220,16000015,sniper,very long,normal,3360,n,0.8360,0.1640
23965,MORTIS,BO,PIPER,hotZone,Quarter Pounder,hotZone,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000011,assassin,short,very fast,5700,16000014,area control,long,normal,5220,16000015,sniper,very long,normal,3360,n,0.8360,0.1640
23963,MORTIS,BO,PIPER,hotZone,Dueling Beetles,hotZone,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000011,assassin,short,very fast,5700,16000014,area control,long,normal,5220,16000015,sniper,very long,normal,3360,n,0.8360,0.1640
23964,MORTIS,BO,PIPER,hotZone,Ring of Fire,hotZone,ranked,32622.569317,623.282998,11,32622.569317,623.282998,11,32622.569317,623.282998,11,16000011,assassin,short,very fast,5700,16000014,area control,long,normal,5220,16000015,sniper,very long,normal,3360,n,0.8360,0.1640
